In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
import os
import sys
import optuna
import joblib
import pandas as pd
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from datetime import datetime, timedelta
from WattPredictor.utils.helpers import *
from WattPredictor.utils.ts_generator import features_and_target, get_pipeline, average_demand_last_4_weeks
from WattPredictor.config.model_config import ModelConfigurationManager
from WattPredictor.utils.feature import feature_store_instance
from WattPredictor.entity.config_entity import TrainerConfig
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from WattPredictor.utils.exception import CustomException
from WattPredictor.utils.logging import logger

class Trainer:
    def __init__(self, config: TrainerConfig):
        self.config = config
        self.feature_store = feature_store_instance()
        self.models = {
            "XGBoost": {
                "search_space": lambda trial: {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                    "max_depth": trial.suggest_int("max_depth", 3, 10),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                }
            },
            "LightGBM": {
                "search_space": lambda trial: {
                    "num_leaves": trial.suggest_int("num_leaves", 20, 150),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                }
            }
        }

    def load_training_data(self):
        try:
            df, _ = self.feature_store.get_training_data("elec_wx_features_view")
            df = df[['date', 'demand', 'sub_region_code', 'temperature_2m', 
                     'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]
            df.sort_values("date", inplace=True)
            logger.info(f"Loaded training data with columns: {list(df.columns)}")
            logger.info(f"Data range: {df['date'].min()} to {df['date'].max()}")
            return df
        except Exception as e:
            raise CustomException(f"Failed to load data from elec_wx_features_view: {e}", sys)

    def train(self):
        try:
            df = self.load_training_data()
            if df.empty:
                raise CustomException("Loaded DataFrame is empty", sys)

            # Validate data per sub_region_code
            for code in df['sub_region_code'].unique():
                count = len(df[df['sub_region_code'] == code])
                if count < self.config.input_seq_len + 1:
                    logger.warning(f"sub_region_code {code} has insufficient data points ({count} < {self.config.input_seq_len + 1})")

            cutoff_date = (datetime.now() - timedelta(days=90)).strftime("%Y-%m-%d")
            train_df, test_df = df[df['date'] < cutoff_date], df[df['date'] >= cutoff_date]

            logger.info(f"train_df shape: {train_df.shape}, date range: {train_df['date'].min()} to {train_df['date'].max()}")
            logger.info(f"test_df shape: {test_df.shape}, date range: {test_df['date'].min()} to {test_df['date'].max()}")

            if train_df.empty:
                raise CustomException("Training DataFrame is empty after applying cutoff_date", sys)
            if test_df.empty:
                raise CustomException("Test DataFrame is empty after applying cutoff_date", sys)

            train_x, train_y = features_and_target(train_df, self.config.input_seq_len, self.config.step_size)
            train_x.drop(columns=["date"], errors="ignore", inplace=True)

            # Validate dtypes
            non_numeric_cols = train_x.select_dtypes(exclude=['int64', 'float64', 'bool']).columns
            if not non_numeric_cols.empty:
                raise CustomException(f"Non-numeric columns found in train_x: {non_numeric_cols}", sys)

            # Validate expected features
            expected_features = [f'demand_previous_{i+1}_hour' for i in reversed(range(self.config.input_seq_len))] + \
                                ['temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']
            missing_features = [col for col in expected_features if col not in train_x.columns]
            if missing_features:
                logger.warning(f"Missing expected features in train_x: {missing_features}. Proceeding with available features.")

            # Preprocess train_x for input schema
            train_x_transformed = train_x.copy()
            train_x_transformed = average_demand_last_4_weeks(train_x_transformed)

            best_overall = {"model_name": None, "score": float("inf"), "params": None}

            for model_name, model_info in self.models.items():
                logger.info(f"Running Optuna HPO for {model_name}")

                def objective(trial):
                    params = model_info["search_space"](trial)
                    pipeline = get_pipeline(model_type=model_name, **params)
                    x_tr, x_val, y_tr, y_val = train_test_split(train_x, train_y, test_size=0.2, shuffle=False)
                    pipeline.fit(x_tr, y_tr)
                    preds = pipeline.predict(x_val)
                    return mean_squared_error(y_val, preds)

                study = optuna.create_study(direction="minimize")
                study.optimize(objective, n_trials=self.config.n_trials)

                best_params = study.best_params
                pipeline = get_pipeline(model_type=model_name, **best_params)
                score = -cross_val_score(pipeline, train_x, train_y, cv=KFold(n_splits=self.config.cv_folds), 
                                        scoring="neg_root_mean_squared_error").mean()

                if score < best_overall["score"]:
                    best_overall.update({
                        "model_name": model_name,
                        "score": score,
                        "params": best_params
                    })

            final_pipeline = get_pipeline(model_type=best_overall["model_name"], **best_overall["params"])
            final_pipeline.fit(train_x, train_y)

            model_path = Path(self.config.root_dir) / self.config.model_name
            create_directories([model_path.parent])
            save_bin(final_pipeline, model_path)

            input_schema = Schema(train_x_transformed.head(10))
            output_schema = Schema(pd.DataFrame(train_y))
            model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

            model_registry = self.feature_store.project.get_model_registry()
            hops_model = model_registry.python.create_model(
                name=f"wattpredictor_{best_overall['model_name'].lower()}",
                metrics={'rmse': best_overall["score"]},
                input_example=train_x_transformed.head(10),
                model_schema=model_schema,
                description=f"Best {best_overall['model_name']} model with feature engineering pipeline for electricity demand"
            )
            hops_model.save(model_path.as_posix())

            logger.info(f"Best model registered: {best_overall['model_name']} with RMSE {best_overall['score']}, params {best_overall['params']}")
            return best_overall

        except Exception as e:
            raise CustomException(f"Training failed: {e}", sys)

In [3]:
try:    
    config = ModelConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = Trainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise CustomException(e, sys) from e

[2025-07-20 18:44:46,994: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-20 18:44:46,999: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-20 18:44:47,003: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-20 18:44:47,004: INFO: helpers: created directory at: artifacts]
[2025-07-20 18:44:47,005: INFO: helpers: created directory at: artifacts/trainer]
[2025-07-20 18:44:47,014: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-20 18:44:47,017: INFO: external: Initializing external client]
[2025-07-20 18:44:47,018: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-20 18:44:49,902: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240214
[2025-07-20 18:44:52,592: INFO: feature_store: Connected to Hopsworks Feature Store: WattPredictor]
Finished: Reading data from Hopsworks, using Hopswor

Generating TS features:   0%|          | 0/11 [00:00<?, ?it/s]

[2025-07-20 18:45:08,512: INFO: ts_generator: Columns for sub_region_code 7: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]


Generating TS features:   9%|▉         | 1/11 [00:00<00:02,  4.00it/s]

[2025-07-20 18:45:08,761: INFO: ts_generator: Columns for sub_region_code 6: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]


...

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading f:\WattPredictor\artifacts/trainer/model.joblib: 0.000%|          | 0/1590937 elapsed<00:00 remainin…

Uploading f:\WattPredictor\input_example.json: 0.000%|          | 0/4744 elapsed<00:00 remaining<?

Uploading f:\WattPredictor\model_schema.json: 0.000%|          | 0/61796 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1240214/models/wattpredictor_lightgbm/2
[2025-07-20 18:51:00,427: INFO: 2453969390: Best model registered: LightGBM with RMSE 497.51533190325415, params {'num_leaves': 69, 'learning_rate': 0.04171679963012042, 'n_estimators': 256}]
